# Bayesian Optimization
Idea: Use bayesian optimization for (wrapper) feature selection.

In [2]:
from sklearn import datasets
from sklearn import svm
import pandas as pd
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFECV
from bayes_opt import BayesianOptimization
from itertools import chain, combinations

## 1. Choose dataset

In [3]:
# regression dataset
dataset = datasets.load_boston()

In [32]:
# classification dataset
dataset = datasets.load_breast_cancer()

## 2. Import datasets

In [4]:
# define which dataset should be used
data = dataset.data
data_target = dataset.target
feature_names = dataset.feature_names

data_frame = pd.DataFrame(data, columns = feature_names)
data_frame['Target'] = data_target
X = data_frame.drop("Target", 1)       # feature matrix
y = data_frame['Target'] 
data_frame.head()

#print(data)
#print(target_matrix)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,Target
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


## 3. Forward selection Algorithm

In [7]:
sfs = SFS(LinearRegression(),
          k_features=11,
          forward=True,
          floating=False,
          scoring = 'r2',
          cv = 0)

In [9]:
sfs.fit(X, y)
#sfs.k_feature_names_ 

SequentialFeatureSelector(cv=0, estimator=LinearRegression(), k_features=11,
                          scoring='r2')

# TODO from here:

## 4. Create feature-subsets (wrapper)

In [6]:
s = feature_names 
wrapper = list(chain.from_iterable(combinations(s, r) for r in range(len(s)+1)))
#print(wrapper)

## 5. Bayesian Optimization Algorithm

In [21]:
def black_box_function(x):
    #todo
    return wrapper[x]


In [29]:
print(black_box_function(1))

('CRIM',)


In [27]:
 pbounds = {'x': (1, len(wrapper))}s

In [15]:
optimizer = BayesianOptimization(
    f=black_box_function,
    pbounds=pbounds,
    verbose=2, # verbose = 1 prints only when a maximum is observed, verbose = 0 is silent
    random_state=1,
)

In [16]:
optimizer.maximize(
    init_points=2,
    n_iter=3,
)

|   iter    |  target   |     x     |
-------------------------------------
|  1        |  1.0      |  3.416e+0 |
|  2        |  1.0      |  5.901e+0 |


ValueError: array must not contain infs or NaNs